In [ ]:
import sklearn
import seaborn as sns
import pandas as pd
import statistics as stat
from copy import deepcopy
from random import randint

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, scale, PolynomialFeatures, OneHotEncoder, OrdinalEncoder, RobustScaler
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier

from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score

from xgboost import XGBClassifier

from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, KMeansSMOTE, SVMSMOTE
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours, CondensedNearestNeighbour, RepeatedEditedNearestNeighbours, RandomUnderSampler

# Configuration
pd.set_option('display.max_columns', 40) # Hacer que pandas muestre 40 columnas de un dataframe

---

<center><h2>Cargar Dataset</h2></center>

---

In [2]:
URL_TRAIN_CSV = "https://gitlab.com/villainnumber1/filesml_proyecto_2/-/raw/main/train_clean.csv"
URL_TEST_CSV = "https://gitlab.com/villainnumber1/filesml_proyecto_2/-/raw/main/test_clean.csv"
#URL_TRAIN_CSV = "csv/train_clean.csv"
#URL_TEST_CSV = "csv/test_clean.csv"

X = pd.read_csv(URL_TRAIN_CSV, sep=",")
Y = pd.read_csv(URL_TEST_CSV, sep=",")
#X

NameError: ignored

In [ ]:
X

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,43161,3985,4,Lawrencez,40.0,856431180.0,Accountant,97384.44,8167.370000,1.0,3.0,8.0,1.0,Debt Consolidation Loan,20.0,12.0,0.52,3.0,Good,405.45,32.643281,336.0,No,77.284943,53.714219,High_spent_Large_value_payments,847.172909,Standard
1,40663,49494,2,Mohammedr,39.0,417453942.0,Writer,31369.58,2607.131667,8.0,9.0,27.0,8.0,"Home Equity Loan, Not Specified, Not Specified...",30.0,25.0,19.70,6.0,Bad,4259.91,39.375507,144.0,Yes,171.748418,44.990811,High_spent_Large_value_payments,280.855383,Standard
2,111059,2925,6,Alistairn,17.0,499528888.0,Writer,30171.86,2772.321667,6.0,7.0,30.0,9.0,"Personal Loan, Home Equity Loan, Home Equity L...",49.0,18.0,23.49,10.0,Bad,3406.51,34.039021,123.0,Yes,205.403926,40.781489,High_spent_Large_value_payments,247.322615,Standard
3,72139,21358,2,Dinesh Naire,22.0,859449371.0,Doctor,13100.02,1246.668333,7.0,5.0,6.0,5.0,"Credit-Builder Loan, Credit-Builder Loan, Not ...",6.0,20.0,10.64,8.0,Standard,727.29,34.747262,129.0,Yes,41.177132,23.005463,Low_spent_Large_value_payments,298.156832,Standard
4,119841,21221,4,Bakerg,26.0,485239890.0,Doctor,62313.54,5120.795000,6.0,2.0,18.0,4.0,"Auto Loan, Payday Loan, Student Loan, and Cred...",27.0,8.0,12.82,9.0,Standard,817.64,24.545394,122.0,NM,196.679642,63.346750,Low_spent_Medium_value_payments,152.669902,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,42636,39869,7,Hendrik Sackmannb,50.0,217529489.0,Media_Manager,107761.29,9047.107500,4.0,5.0,9.0,0.0,No Data,7.0,19.0,14.56,3.0,Standard,34.09,41.899992,211.0,Yes,0.000000,150.220682,High_spent_Large_value_payments,984.123010,Standard
79996,114031,38305,2,Alexj,33.0,98861902.0,Scientist,16218.32,1098.526667,6.0,6.0,21.0,5.0,"Mortgage Loan, Home Equity Loan, Credit-Builde...",44.0,19.0,8.98,6.0,Bad,4466.33,24.275710,98.0,NM,64.533114,34.457893,High_spent_Small_value_payments,260.523798,Poor
79997,85991,9976,6,Luciana Lopezi,25.0,745835485.0,Engineer,29374.77,2377.897500,3.0,7.0,6.0,2.0,"Student Loan, and Payday Loan",23.0,13.0,16.65,7.0,Standard,1189.37,38.439021,241.0,NM,40.264074,22.234178,High_spent_Medium_value_payments,403.980355,Standard
79998,21240,13193,7,Edward Krudyd,15.0,894419091.0,Manager,43782.45,3543.537500,6.0,9.0,25.0,8.0,"Payday Loan, Auto Loan, Payday Loan, Home Equi...",32.0,16.0,1.88,12.0,Bad,3468.25,24.233841,85.0,Yes,240.664739,42.839562,Low_spent_Large_value_payments,146.741586,Poor


---

<center><h2>Procesar columnas</h2></center>

---

In [ ]:
normalizer = StandardScaler(with_mean=False)

def unionColumnas(df1, df2, columnName):
  return list(set(df1[columnName]).union(set(df2[columnName])))

# TODO: No hay una manera mejor de hacer esto?
keysAge        = unionColumnas(X, Y, "Age")
keysOccupation = unionColumnas(X, Y, "Occupation")
keysPMA        = unionColumnas(X, Y, "Payment_of_Min_Amount")
keysCM         = unionColumnas(X, Y, "Credit_Mix")
keysPB         = unionColumnas(X, Y, "Payment_Behaviour")
keysNBA        = unionColumnas(X, Y, "Num_Bank_Accounts")
keysNCC        = unionColumnas(X, Y, "Num_Credit_Card")
keysIR         = unionColumnas(X, Y, "Interest_Rate")
keysNOL        = unionColumnas(X, Y, "Num_of_Loan")
keysIR         = unionColumnas(X, Y, "Interest_Rate")
keysCM         = unionColumnas(X, Y, "Credit_Mix")
keysPMA        = unionColumnas(X, Y, "Payment_of_Min_Amount")
keysNDP        = unionColumnas(X, Y, "Num_of_Delayed_Payment")
keysNCI        = unionColumnas(X, Y, "Num_Credit_Inquiries")
keysCHA        = unionColumnas(X, Y, "Credit_History_Age")



# Transformar las columnas
# TODO: Considerar ensembles para mismos clientes?
# TODO: Considerar OrdinalEncoder para features del estilo Good-Normal-Bad
transformersX = [
  ("ID",                                              "drop",                                                       [0]),    # Por razones obvias
  ("Customer_ID",                                     "passthrough",                                                [1]),    # Por razones obvias
  ("Month",                                           "passthrough",                                                [2]),    # No creo que tenga ninguna influencia
  ("Name",                                            "drop",                                                       [3]),    # No tiene influencia y está repetida
  ("Age",                                             "drop",                                                       [4]),
  ("SSN",                                             "drop",                                                       [5]),    # No tiene influencia y está repetida
  ("Occupation",                                      OneHotEncoder(categories=[keysOccupation]),                   [6]),    # No deberia influir, pero baja un poco el error? TODO: Comprobar
  ("Annual_Income",                                   "passthrough",                                                [7]),
  ("Monthly_Inhand_Salary",                           "drop",                                                       [8]),    # Annual_Income/12
  ("Num_Bank_Accounts",                               OneHotEncoder(categories=[keysNBA]),                          [9]),    # No tiene influencia
  ("Num_Credit_Card",                                 OneHotEncoder(categories=[keysNCC]),                          [10]),
  ("Interest_Rate",                                   OneHotEncoder(categories=[keysIR]),                           [11]),
  ("Num_of_Loan",                                     OneHotEncoder(categories=[keysNOL]),                          [12]),
  ("Type_of_Loan",                                    "drop",                                                       [13]),   # Dividida
  ("Delay_from_due_date",                             "passthrough",                                                [14]),
  ("Num_of_Delayed_Payment",                          OneHotEncoder(categories=[keysNDP]),                          [15]),
  ("Changed_Credit_Limit",                            "passthrough",                                                [16]),
  ("Num_Credit_Inquiries",                            OneHotEncoder(categories=[keysNCI]),                          [17]),
  ("Credit_Mix",                                      OneHotEncoder(categories=[keysCM]),                           [18]),
  ("Outstanding_Debt",                                "passthrough",                                                [19]),
  ("Credit_Utilization_Ratio",                        "passthrough",                                                [20]),
  ("Credit_History_Age",                              OneHotEncoder(categories=[keysCHA]),                          [21]),
  ("Payment_of_Min_Amount",                           OneHotEncoder(categories=[keysPMA]),                          [22]),   # TODO: Considerar binary encoding
  ("Total_EMI_per_Month",                             "passthrough",                                                [23]),
  ("Amount_Invested_Monthly",                         "passthrough",                                                [24]),   # No tiene influencia
  ("Payment_Behaviour",                               OneHotEncoder(categories=[keysPB]),                           [25]),
  ("Monthly_Balance",                                 "passthrough",                                                [26]),
  ("Credit_Score",                                    "drop",                                                       [27]),

  # Type_of_Loan
  ("_Auto_Loans",                                     "passthrough",                                                [28]),
  ("_Credit-Builder_Loans",                           "passthrough",                                                [29]),
  ("_Debt_Consolidation_Loans",                       "passthrough",                                                [30]),
  ("_Home_Equity_Loans",                              "passthrough",                                                [31]),
  ("_Mortgage_Loans",                                 "passthrough",                                                [32]),
  ("_Payday_Loans",                                   "passthrough",                                                [33]),
  ("_Personal_Loans",                                 "passthrough",                                                [34]),
  ("_Student_Loans",                                  "passthrough",                                                [35])
]
    
transformersY = deepcopy(transformersX)

# Normbrar el orden de columnas de 0 a n-1
transformersY.pop(27) # Eliminar Credit_Score
for i, t in enumerate(transformersY): # Renumerar los indices de las columnas
  transformersY[i][2].pop()
  transformersY[i][2].append(i)

X_T = ColumnTransformer(transformers=transformersX).fit_transform(X)
Y_T = ColumnTransformer(transformers=transformersY).fit_transform(Y)

print(X_T.shape)
print(Y_T.shape)

######################
# Normalize the data #
######################
X_T = normalizer.fit_transform(X_T)
Y_T = normalizer.fit_transform(Y_T)

ValueError: ignored

---

<center><h2>Sampling</h2></center>

---

In [ ]:
_X_T = X_T
_Credit_Score = X["Credit_Score"]

# Tomek links
resampler = TomekLinks(sampling_strategy=["Standard"], n_jobs=-1)
_X_T, _Credit_Score = resampler.fit_resample(_X_T, _Credit_Score)

#resampler = CondensedNearestNeighbour() # No termina nunca.
#resampler = RepeatedEditedNearestNeighbours(sampling_strategy="not majority", n_jobs=-1, n_neighbors=1)
#resampler = RandomUnderSampler(sampling_strategy="majority")

#_X_T, _Credit_Score = resampler.fit_resample(_X_T, _Credit_Score)

# SMOTE
#resampler = SMOTE(sampling_strategy={"Poor": 12800, "Good": 36000}, random_state=42, k_neighbors=5)

#_X_T, _Credit_Score = resampler.fit_resample(_X_T, _Credit_Score)

In [ ]:
print("Poor     :", len([x for x in _Credit_Score if x == "Poor"]))     # Before: 23199
print("Standard :", len([x for x in _Credit_Score if x == "Standard"])) # Before: 42539
print("Good     :", len([x for x in _Credit_Score if x == "Good"]))     # Before: 14262

---

<center><h2>Score</h2></center>

---

In [ ]:
scoring = {
    "precision_micro" : make_scorer(precision_score, average = 'micro'),
    "recall_micro"    : make_scorer(recall_score,    average = 'micro'),
    "precision_macro" : make_scorer(precision_score, average = 'macro'),
    "recall_macro"    : make_scorer(recall_score,    average = 'macro'),
    "f1_micro"        : make_scorer(f1_score,        average = 'micro'),
    "f1_macro"        : make_scorer(f1_score,        average = 'macro')
}

def print_detailed(scores, cv):
    # Añadir medias
    precision_micro = scores["test_precision_micro"].tolist() + [stat.mean(scores["test_precision_micro"]), ""]
    recall_micro    = scores["test_recall_micro"].tolist()    + [stat.mean(scores["test_recall_micro"]),    ""]
    precision_macro = scores["test_precision_macro"].tolist() + [stat.mean(scores["test_precision_macro"]), ""]
    recall_macro    = scores["test_recall_macro"].tolist()    + [stat.mean(scores["test_recall_macro"]),    ""]
    f1_micro        = scores["test_f1_micro"].tolist()        + [stat.mean(scores["test_f1_micro"]),  stat.variance(scores["test_f1_micro"])**(1/2)]
    f1_macro        = scores["test_f1_macro"].tolist()        + [stat.mean(scores["test_f1_macro"]),  stat.variance(scores["test_f1_macro"])**(1/2)]
    
    rf_score = pd.DataFrame()
    rf_score.insert(0, "precision_micro", precision_micro)
    rf_score.insert(1, "recall_micro",    recall_micro)
    rf_score.insert(2, "f1_micro",        f1_micro)
    rf_score.insert(3, "precision_macro", precision_macro)
    rf_score.insert(4, "recall_macro",    recall_macro)
    rf_score.insert(5, "f1_macro",        f1_macro)
    
    rf_score.index = [str(n) for n in range(cv)] + ["avg", "std"]
    
    display(rf_score)

In [ ]:
#clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced")

#scores = cross_validate(clf, _X_T, _Credit_Score, cv=5, n_jobs=-1, scoring=scoring)
#print_detailed(scores, cv=5)

---

<center><h2>Entrenamiento y Prediccion</h2></center>

---

In [ ]:
#%%script false --no-raise-error

clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
#clf = XGBClassifier(seed=42)

clf.fit(_X_T, _Credit_Score)
pred = clf.predict(Y_T)

---

<center><h2>CSV</h2></center>

---

In [ ]:
#%script false --no-raise-error

solucion = pd.DataFrame(data = { "ID" : Y["ID"], "Credit_Score": pred })
solucion.to_csv("solucion.csv", index = False)

---

<center><h2>Pruebas</h2></center>

---

In [ ]:
%%script false --no-raise-error

##################
# RandomsearchCV #
##################

import random

best_score = 0
best_params = None

def take_n(n, rang):
    l = list(rang)
    ret = []
    
    for i in range(n):
        rand = random.choice(l)
        l.remove(rand)
        ret.append(rand)
    
    return ret

n = 2
#n_estimators     = take_n(n, range(100, 400, 50))
min_samples_leaf = take_n(n, range(1, 7, 2))
max_depths       = take_n(n, range(10, 120, 5))

for min_samples_leaf in min_samples_leaf:
    for max_depth in max_depths:
        clf = RandomForestClassifier(n_estimators=100, min_samples_leaf=min_samples_leaf, max_depth=max_depth, random_state=42, n_jobs=-1, class_weight="balanced")
        scores = cross_validate(clf, _X_T, _Credit_Score, cv=5, n_jobs=-1, scoring=scoring)

        mean = stat.mean(scores["test_f1_micro"])

        if mean > best_score:
            best_score = mean
            best_params = [n_estimator, min_samples_leaf, max_depth]

            print(best_score, best_params)

print("end")

In [ ]:
%%script false --no-raise-error

for strategy in ["majority", "not majority", "not minority", "all"]:
    print(strategy)
    
    _X_T = X_T
    _Credit_Score = X["Credit_Score"]

    # Tomek links
    resampler = TomekLinks(sampling_strategy=strategy, n_jobs=-1)
    _X_T, _Credit_Score = resampler.fit_resample(_X_T, _Credit_Score)

    clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced")

    scores = cross_validate(clf, _X_T, _Credit_Score, cv=5, n_jobs=-1, scoring=scoring)
    print_detailed(scores, cv=5)
    print("Poor     :", len([x for x in _Credit_Score if x == "Poor"]))     # Before: 23199
    print("Standard :", len([x for x in _Credit_Score if x == "Standard"])) # Before: 42539
    print("Good     :", len([x for x in _Credit_Score if x == "Good"]))     # Before: 14262
    print()

In [ ]:
for strategyTomek in ["majority", "not majority", "not minority", "all"]:
    for strategySmote in ["minority", "not majority", "not minority", "all"]:
        print(strategyTomek, strategySmote)
        
        _X_T = X_T
        _Credit_Score = X["Credit_Score"]
        clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

        # Tomek links
        resampler = TomekLinks(sampling_strategy=strategyTomek, n_jobs=-1)
        _X_T, _Credit_Score = resampler.fit_resample(_X_T, _Credit_Score)

        # SMOTE
        resampler = SMOTE(sampling_strategy=strategySmote, random_state=42)
        _X_T, _Credit_Score = resampler.fit_resample(_X_T, _Credit_Score)

        scores = cross_validate(clf, _X_T, _Credit_Score, cv=5, n_jobs=-1, scoring=scoring)
        print_detailed(scores, cv=5)
        print("Poor     :", len([x for x in _Credit_Score if x == "Poor"]))     # Before: 23199
        print("Standard :", len([x for x in _Credit_Score if x == "Standard"])) # Before: 42539
        print("Good     :", len([x for x in _Credit_Score if x == "Good"]))     # Before: 14262
        print()